[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Cell Segmentation Using YOLO**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## Overview

This notebook demonstrates a complete workflow for segmenting and analyzing cells in microscopy video using the YOLO (You Only Look Once) segmentation model.

#### Real-World Applications:
- Cell culture analysis  
- Automated microscopy workflows  
- Biomedical image processing  
- Quantifying cell morphology and movement over time

## Annotate your Custom dataset using Labellerr

 ***1. Visit the [Labellerr](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) website and click **“Sign Up”**.*** 

 ***2. After signing in, create your workspace by entering a unique name.***

 ***3. Navigate to your workspace’s API keys page (e.g., `https://<your-workspace>.labellerr.com/workspace/api-keys`) to generate your **API Key** and **API Secret**.***

 ***4. Store the credentials securely, and then use them to initialise the SDK or API client with `api_key`, `api_secret`.*** 


### Use Labellerr SDK for uploading and perform annotation of your own dataset

In [1]:
# uncomment the following lines to install required packages in a Jupyter notebook environment

# !pip install git+https://github.com/Labellerr/SDKPython.git
# !pip install ipyfilechooser
# !git clone https://github.com/Labellerr/yolo_finetune_utils.git

In [ ]:
# all the imports required for this notebook
from labellerr.client import LabellerrClient
from labellerr.core.datasets import create_dataset_from_local
from labellerr.core.annotation_templates import create_template
from labellerr.core.projects import create_project
from labellerr.core.schemas import DatasetConfig, AnnotationQuestion, QuestionType, CreateTemplateParams, DatasetDataType, CreateProjectParams, RotationConfig
from labellerr.core.projects import LabellerrProject
from labellerr.core.exceptions import LabellerrError

import uuid
from ipyfilechooser import FileChooser

In [ ]:
api_key = input("YOUR_API_KEY")        # go to labellerr workspace to get your API key
api_secret = input("YOUR_API_SECRET")  # go to labellerr workspace to get your API secret
client_id = input("YOUR_CLIENT_ID")   # Contact labellerr support to get your client ID i.e. support@tensormatics.com

client = LabellerrClient(api_key, api_secret, client_id)


### ***STEP-1: Create a dataset on labellerr from your local folder***

The SDK supports in creating dataset by uploading local files. 

In [ ]:
# Create a folder chooser starting from a directory (for example, your home directory)
chooser = FileChooser('/')

# Set the chooser to folder selection mode only
chooser.title = 'Select a folder containing your dataset'
chooser.show_only_dirs = True

# Display the widget
display(chooser)

FileChooser(path='D:\', filename='', title='Select a folder containing your dataset', show_hidden=False, selec…

In [ ]:
path_to_dataset = chooser.selected_path
print("You selected:", path_to_dataset)

You selected: D:\Professional\Projects\Cell_Segmentation_using_YOLO\frames_output


In [ ]:
my_dataset_type = input("Enter your dataset type (video or image): ").lower()
print("Selected dataset type:", my_dataset_type)

Selected dataset type: video


In [ ]:
dataset = create_dataset_from_local(
    client=client,
    dataset_config=DatasetConfig(dataset_name="My Dataset", data_type="image"),
    folder_to_upload=path_to_dataset
)

print(f"Dataset created with ID: {dataset.dataset_id}")


### ***STEP-2: Create annotation project on labellerr of your created dataset***

Create a annotation project of your uploaded dataset to start performing annotation on labellerr UI

In [ ]:
# Create annotation guideline template for video annotation project (like classes to be annotated)

template = create_template(
    client=client,
    params=CreateTemplateParams(
        template_name="My Template",
        data_type=DatasetDataType.image,
        questions=[
            AnnotationQuestion(
                question_number=1,
                question="Object",
                question_id=str(uuid.uuid4()),
                question_type=QuestionType.polygon,
                required=True,
                color="#FF0000"
            )
        ]
    )
)
print(f"Annotation template created with ID: {template.annotation_template_id}")


In [ ]:
dataset.status()        # wait until dataset is processed before creating project

project = create_project(
    client=client,
    params=CreateProjectParams(
        project_name="My Project",
        data_type=DatasetDataType.image,
        rotations=RotationConfig(
            annotation_rotation_count=1,
            review_rotation_count=1,
            client_review_rotation_count=1
        )
    ),
    datasets=[dataset],
    annotation_template=template
)

print(f"✓ Project created: {project.project_id}")

Your project has been created now go to labellerr platform to perform annotation 

***click to go to labellerr.com***

[![Labellerr](https://cdn.labellerr.com/1%20%20Documentation/1c9dc7ce-9a54-4111-8fd5-0363ba3e00e1.webp)](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks)
Open the project you created (Projects → select your project).

Click Start Labeling to open the annotation interface. Use the configured labeling tools (bounding boxes, polygon, dot, classification, etc.) to annotate files.
### ***STEP-3: Export your annotation in required format***

Generate a temporary download URL to retrieve your exported JSON file:

### Export Configuration Parameters

| Parameter | Type | Description |
|-----------|------|-------------|
| `export_name` | string | Display name for the export |
| `export_description` | string | Description of what this export contains |
| `export_format` | string | Output format (e.g., `json`, `xml`, `coco`) |
| `statuses` | list | Annotation statuses to include in export |

### Common Annotation Statuses

- **`review`**: Annotations pending review
- **`r_assigned`**: Review assigned to a reviewer
- **`client_review`**: Under client review
- **`cr_assigned`**: Client review assigned
- **`accepted`**: Annotations accepted and finalized

---

In [ ]:
export_config = {
    "export_name": "Weekly Export",
    "export_description": "Export of all accepted annotations",
    "export_format": "coco_json",
    "statuses": ['review', 'r_assigned','client_review', 'cr_assigned','accepted']
}

try:
    # Get project instance
    project = LabellerrProject(client=client, project_id=project.project_id)
    
    # Create export
    result = project.create_local_export(export_config)
    export_id = result["response"]['report_id']
    print(f"Local export created successfully. Export ID: {export_id}")
except LabellerrError as e:
    print(f"Local export creation failed: {str(e)}")
    
    
try:
    download_url = client.fetch_download_url(
        project_id=project.project_id,
        uuid=str(uuid.uuid4()),
        export_id=export_id
    )
    print(f"Download URL: {download_url}")
except LabellerrError as e:
    print(f"Failed to fetch download URL: {str(e)}")


Now you can download your annotations locally using given URL

# COCO to YOLO Format Conversion

Converts COCO-style segmentation annotations to YOLO segmentation dataset format.  
- Requires: `annotation.json` and images in `frames_output` directory.
- Output: Generated YOLO dataset folder.
- Parameters: allows train/val split, shuffling, and verbose mode.


In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

ANNOTATION_JSON = r"annotation.json"
IMAGE_DIR = r"image_dataser_dir"


coco_to_yolo_converter(
        json_path=ANNOTATION_JSON,
        images_dir=IMAGE_DIR,
        output_dir="yolo_dataset",
        use_split=False,
        train_ratio=0.8,
        val_ratio=0.2,
        test_ratio=0.0,
        shuffle=True,
        verbose=False
    )

# Clear CUDA Cache

Frees up GPU memory by clearing unused cached memory before starting new tasks or model training.


In [ ]:
import torch
torch.cuda.empty_cache()

# Check GPU Memory Status

Prints the GPU memory currently allocated, cached, and free.  
Useful for diagnosing memory usage before training or inference.


In [ ]:
# Check GPU memory status
print(f"Allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")
print(f"Free: {torch.cuda.mem_get_info(0)[0]/1024**3:.2f} GB")

# Load and Train YOLO Segmentation Model

Loads the YOLO segmentation model and trains it using the converted YOLO dataset.
- Data: Path to YOLO-style `data.yaml`
- Parameters: epochs, image size, batch size, device, dataloader workers, experiment name.


In [ ]:
from ultralytics import YOLO
# Load a model
model = YOLO("yolo11m-seg.pt")

# Train the model
results = model.train(
    data=r"yolo_dataset\data.yaml",    # Path to your dataset YAML file
    epochs=500,                        # Number of training epochs
    imgsz=640,                         # Image size
    batch=-1,                          # Batch size
    device=0,                          # GPU device (0 for first GPU, 'cpu' for CPU)
    workers=4,                         # Number of dataloader workers
    name="Model m"
)

# CellSegmentationVideo Class Definition

Defines the `CellSegmentationVideo` class for performing video cell segmentation using YOLO.
- Initialization requires YOLO model path, confidence threshold, IOU threshold.
- Contains methods for polygon area calculation, mask center localization, frame processing, and video processing.
- Tracks cells, draws masks and borders, labels areas and track IDs.


In [ ]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

class CellSegmentationVideo:
    def __init__(self, model_path, conf_threshold=0.25, iou_threshold=0.7):
        """
        Initialize the cell segmentation video inference class.
        
        Args:
            model_path: Path to trained YOLO segmentation model
            conf_threshold: Confidence threshold for detections
            iou_threshold: IOU threshold for NMS
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        
        # Generate distinct colors for different tracked objects
        np.random.seed(42)
        self.colors = [tuple(map(int, np.random.randint(0, 255, 3))) 
                       for _ in range(100)]
        
        # Track history for smoothing
        self.track_history = defaultdict(lambda: [])
        
    def calculate_polygon_area(self, points):
        """Calculate area of polygon using Shoelace formula."""
        x = points[:, 0]
        y = points[:, 1]
        return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    
    def get_polygon_center(self, points):
        """Calculate center of polygon."""
        moments = cv2.moments(points)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])
            return (cx, cy)
        return (int(np.mean(points[:, 0])), int(np.mean(points[:, 1])))
    
    def process_frame(self, frame):
        """
        Process a single frame with cell segmentation.
        
        Args:
            frame: Input frame (BGR format)
            
        Returns:
            Annotated frame with colored segmentation masks and areas
        """
        # Run tracking (built-in YOLO tracker)
        results = self.model.track(
            frame, 
            persist=True,
            conf=self.conf_threshold,
            iou=self.iou_threshold,
            verbose=False
        )
        
        annotated_frame = frame.copy()
        
        if results[0].masks is not None:
            masks = results[0].masks.xy
            boxes = results[0].boxes
            
            # Get track IDs if available
            if boxes.id is not None:
                track_ids = boxes.id.int().cpu().tolist()
            else:
                track_ids = list(range(len(masks)))
            
            for mask, track_id in zip(masks, track_ids):
                if len(mask) == 0:
                    continue
                
                # Get color for this track
                color = self.colors[track_id % len(self.colors)]
                
                # Convert mask to integer coordinates
                mask_points = mask.astype(np.int32)
                
                # Draw filled polygon with transparency
                overlay = annotated_frame.copy()
                cv2.fillPoly(overlay, [mask_points], color)
                cv2.addWeighted(overlay, 0.4, annotated_frame, 0.6, 0, annotated_frame)
                
                # Draw solid border
                cv2.polylines(annotated_frame, [mask_points], True, color, 2)
                
                # Calculate area (in pixels squared)
                area = self.calculate_polygon_area(mask_points)
                
                # Get center point
                center = self.get_polygon_center(mask_points)
                
                # Display area at center
                area_text = f"{area:.0f} px sq"
                
                # Get text size for background
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                thickness = 1
                (text_w, text_h), baseline = cv2.getTextSize(
                    area_text, font, font_scale, thickness
                )
                
                # Draw background rectangle for text
                cv2.rectangle(
                    annotated_frame,
                    (center[0] - text_w//2 - 5, center[1] - text_h//2 - 5),
                    (center[0] + text_w//2 + 5, center[1] + text_h//2 + 5),
                    (255, 255, 255),
                    -1
                )
                
                # Draw area text
                cv2.putText(
                    annotated_frame,
                    area_text,
                    (center[0] - text_w//2, center[1] + text_h//2),
                    font,
                    font_scale,
                    (0, 0, 0),
                    thickness
                )
                
                # Optional: Draw track ID
                cv2.putText(
                    annotated_frame,
                    f"ID:{track_id}",
                    (center[0] - text_w//2, center[1] - text_h//2 - 10),
                    font,
                    0.4,
                    color,
                    1
                )
        
        return annotated_frame
    
    def process_video(self, input_path, output_path, display=True):
        """
        Process entire video file.
        
        Args:
            input_path: Path to input video
            output_path: Path to save output video
            display: Whether to display video while processing
        """
        cap = cv2.VideoCapture(input_path)
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Setup video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        
        print(f"Processing video: {total_frames} frames at {fps} FPS")
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Process frame
            annotated_frame = self.process_frame(frame)
            
            # Write to output
            out.write(annotated_frame)
            
            # Display if requested
            if display:
                cv2.imshow('Cell Segmentation', annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            frame_count += 1
            if frame_count % 30 == 0:
                print(f"Processed {frame_count}/{total_frames} frames")
        
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        
        print(f"Video saved to: {output_path}")



# Specify Model Path

Specifies the file path to the trained YOLO segmentation model weights.  
Update this path as needed to match your environment.


In [ ]:
model_path = r"D:\Professional\Projects\Cell_Segmentation_using_YOLO\runs\segment\Model m\weights\last.pt"

# Example Usage—Run Segmentation on a Video

Demonstrates how to use the `CellSegmentationVideo` class:
- Initializes the processor.
- Runs segmentation inference on a sample video.
- Saves the output and optionally displays annotated frames.


In [ ]:
# Example usage
if __name__ == "__main__":
    # Initialize the segmentation processor
    segmentor = CellSegmentationVideo(
        model_path=model_path,
        conf_threshold=0.5,
        iou_threshold=0.3
    )
    
    # Process a video file
    segmentor.process_video(
        input_path=r"D:\Professional\Projects\Cell_Segmentation_using_YOLO\video\cell segmentation_bw.mp4",
        output_path="output_segmented_2.mp4",
        display=False
    )
    

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
